### COVID-19: Simulación de contagio

Las herramientas computacionales son una gran ayuda al momento de reproducir simulaciones y poder evaluar distintos escenarios. Actualmente estamos transitando un periodo de mucha incertidumbre debido a la pandemia COVID-19. Entender cómo y cuánto podremos aplanar la curva de contagios y dilatarla en el tiempo, para no colapsar las camas de UCI, los respiradores y el personal de salud, es uno de los mayores interrogantes para gobernantes y directores de los centros de salud. En este contexto, poder contar con información actualizada y modelos que puedan inferir los contagios de los próximos días, son un recurso indispensable para poder prevenir y estar preparados.

En este ejercicio, tomaremos uno de los modelos epidemiológicos más simples, capaces de capturar muchas de las características típicas de estos brotes: [el SIR](https://es.wikipedia.org/wiki/Modelo_SIR#:~:text=El%20modelo%20SIR%20es%20uno,t%C3%ADpicas%20de%20los%20brotes%20epid%C3%A9micos.&text=El%20modelo%20relaciona%20las%20variaciones,y%20el%20per%C3%ADodo%20infeccioso%20promedio).  Lo utilizaremos para simular la cantidad de nuevos casos que surgirán en las próximas semanas, en cada una de las comunas de la Ciudad de Buenos Aires.

El nombre del modelo 'SIR' proviene de las iniciales de los tres grupos de individuos que se distinguen por su condición frente a la enfermedad. Estos son:

- Población susceptible (S), individuos sin inmunidad al agente infeccioso, y que por tanto puede ser infectada si es expuesta al agente infeccioso. 
- Población infectada (I), individuos que están infectados en un momento dado y pueden transmitir la infección a individuos de la población susceptible con la que entran en contacto. Para nuestras simulaciones subdividiremos este grupo en dos: llamaremos 'I0' a los que están cursando la primera semana de la enfermedad y 'I1' a quienes están en su segunda semana de la enfermedad y próximos a recibir el alta.
- Población recuperada y fallecidos (R), individuos que son inmunes a la infección (o fallecidos), y consecuentemente no afectan a la transmisión cuando entran en contacto con otros individuos.

El modelo relaciona las variaciones de las tres poblaciones (Susceptible, Infectada y Recuperada) a través de la tasa de infección y el período infeccioso promedio. Es decir, que las personas pasan de susceptibles a infectadas acorde a una tasa de infección que depende del número promedio de contactos por persona por día y la probabilidad de transmisión de la enfermedad en un contacto entre un sujeto susceptible y un infeccioso. Además, las personas pasan de infectadas a recuperadas al cabo del periodo infeccioso promedio. En nuestras simulaciones tomaremos estos dos valores como dados. Usaremos una tasa de infección igual a 0.5 y un período infeccioso igual a dos semanas. Sin embargo, es importante resaltar que en ejercicios de simulación más extensos, estos dos parámetros es importante probarlos y adaptarlos en las simulaciones para que ajusten lo mejor posible a la realidad observada en los datos de las semanas pasadas y en los datos de otras ciudades que ya transitaron esta etapa.

Por ultimo, quisiéramos destacar que, si bien el presente ejercicio esta enmarcado en un posible uso de Python en la vida real, nuestro objetivo es que practiquen los conceptos recientemente aprendidos. Es por ello que el modelo epidemiológico y las simulaciones estarán simplificadas lo suficiente para no perder nuestro objetivo de vista.


In [ ]:
import requests
import pandas as pd
import csv
from datetime import timedelta
import random

In [ ]:
#Definimos constantes que usaremos en el modelo
URL = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/salud/casos-covid-19/casos_covid19.csv"
CSV_NAME = "casos_covid19.csv"
FECHA = "25OCT2020:00:00:00.000000"

def obtengo_infectados_por_comuna(fecha=FECHA, url=URL, csv_name=CSV_NAME):
    '''
    Esta funcion devuelve comuna, genero y edad de las personas infectadas de 
    covid en una determinada fecha
    
    Input:
        fecha (str): fecha de hoy (fecha de ultimos registros actualizados)
        url (str): url de donde descargar el archivo
        csv_name (str): nombre con el cual guardar el archivo descargado
        
    Output:
        Tupla con tres listas de peronas en los tres estados I0, I1, R.
        infectados0 (lista): contiene una tupla por c/persona infectada esta 
            semana. Cada tupla tiene comuna, genero y edad de cada persona.
        infectados1 (lista): contiene una tupla por c/persona infectada esta 
            semana. Cada tupla tiene comuna, genero y edad de cada persona.
        recuperados (lista): contiene una tupla por c/persona infectada esta 
            semana. Cada tupla tiene comuna, genero y edad de cada persona.
    '''
    
    print("Si la función genera error, puede hacer click en el siguiente link \
           \npara iniciar la descarga:", url)

    #Obtenemos el contenido del archivo en el url especificado
    req = requests.get(url)
    url_content = req.content

    #Guardamos el contenido descargado en un archivo con el nombre que elegimos
    csv_file = open(csv_name, 'w')
    csv_file.write(url_content.decode())
    csv_file.close()
    
    #Filtrar la tabla de datos a infectados de CABA 
    #Caja negra por ahora. Cubriremos este contenido en 2 semanas:
    df = pd.read_csv(csv_name)
    df_chico = df[df["provincia"] == "CABA"]
    df_chico = df_chico[pd.notna(df_chico["comuna"])]
    df_chico = df_chico[df_chico["clasificacion"] == "confirmado"]
    df_chico['fecha'] = df_chico['fecha_clasificacion'].\
                        apply(lambda x: pd.to_datetime(x[:9], format='%d%b%Y',
                                                       errors='ignore'))   
    fecha = pd.to_datetime(fecha[:9], format='%d%b%Y')
    
    #Filtramos las personas contagiadas la ultima semana
    df_temp = df_chico[df_chico["fecha"] >= fecha - timedelta(days=7)]
    df_temp = df_temp[["comuna", "genero", "edad"]]
    #Guardamos la lista de personas en estado I0 en una lista de tuplas
    infectados0 = [tuple(x) for x in df_temp.to_numpy()]
    
    #Filtramos las personas contagiadas la semana pasada
    df_temp = df_chico[df_chico["fecha"] >= fecha - timedelta(days=14)]
    df_temp = df_temp[df_temp["fecha"] < fecha - timedelta(days=7)]
    df_temp = df_temp[["comuna", "genero", "edad"]]
    #Guardamos la lista de personas en estado I1 en una lista de tuplas
    infectados1 = [tuple(x) for x in df_temp.to_numpy()]
    
    #Filtramos las personas recuperadas
    df_temp = df_chico[df_chico["fecha"] < fecha - timedelta(days=14)]
    df_temp = df_temp[["comuna", "genero", "edad"]]
    #Guardamos la lista de personas recuperadas en una lista de tuplas
    recuperados = [tuple(x) for x in df_temp.to_numpy()]
    
    return (infectados0, infectados1, recuperados)

#### Ejercicio 1: 
La función obtengo_infectados_por_comuna devuelve tres listas de personas, donde cada persona esta representada con una tupla conteniendo su comuna, su genero y su edad. La primer lista contiene las personas infectadas la ultima semana, la segunda lista contiene las personas infectadas en la semana anterior, y la ultima lista contiene las personas recuperadas. 

Guarden las listas que devuelve obtengo_infectados_por_comuna con los siguientes nombres e impriman una lista para entender mejor su contenido:
- infectados0 
- infectados1
- recuperados

In [ ]:
#TU CODIGO ACA

#### Ejercicio 2:
Creen un diccionario que contenga el numero de la comuna (en formato string) como claves y la suma de personas en cada comuna como valor.

In [ ]:
def suma_personas_por_comuna(lista_de_personas):
    '''
    ESCRIBIR DOC STRING 
    '''
    dic_por_comuna = {}

    #TU CODIGO ACA
    
    return dic_por_comuna   

A continuación usaremos nuestra función suma_personas_por_comuna() sobre las tres listas de personas que tenemos

In [ ]:
i0_por_comuna = suma_personas_por_comuna(infectados0)
i1_por_comuna = suma_personas_por_comuna(infectados1)
r_por_comuna = suma_personas_por_comuna(recuperados)

#### Ejercicio 3:
A continuación, importaremos el archivo con la población en cada comuna de la Ciudad de Buenos Aires y también lo guardaremos como un diccionario con las comunas como claves y el total de población como valores.

In [ ]:
#para leer este archivo importamos el paquete csv que nos va a devolver una
#lista por cada linea del archivo y un elemento en la lista por cada valor 
#separado con coma en el .csv

#Abro el archivo que contiene la población total por comuna (estimación al 2020)
with open('pob_por_comuna.csv', 'r') as f:
    filas = csv.reader(f)
    
    pob_por_comuna = {}
    
    for fila in filas:
        pob_por_comuna[fila[0]] = int(fila[1])
    pob_por_comuna.pop("\ufeffcomuna")
    
pob_total = pob_por_comuna['pob_total']

#eliminen la clave 'pob_total'
#TU CODIGO ACA

pob_por_comuna

#### Ejercicio 4:
A continuación construiremos un diccionario llamado sir_por_comuna donde las claves serán las comunas y los valores serán otro diccionario con los detalles de cada comuna. En los diccionarios con detalle tendremos las siguientes claves 'S', 'I0', 'I1', 'R', 'N' y sus respectivos valores. 

'S': numero de personas susceptible, 

'I0': numero de personas infectadas esta semana, 

'I1': numero de personas infectadas la semana pasada,

'R': numero de personas recuperadas,

'N': total de personas en la comuna

In [ ]:
sir_por_comuna = {}

for comuna in pob_por_comuna.keys():
    sir_por_comuna[comuna] = {
                            #TU CODIGO ACA
                                }


#### Ejercicio 5:
A continuación, usaremos el diccionario sir_por_comuna para crear la representación de cada comuna. Las comunas serán representadas por listas donde cada uno de los elementos será el estado de una persona en dicha comuna. Además, las posiciones adyacentes (una a la izquierda y una a la derecha) de cada elemento de la lista serán considerados los vecinos de cada persona y serán los únicos que pueden contagiar a esa persona (ej. comuna_1 = ['S', 'S', 'R', 'I0', 'I1', 'S'], el vecino en la primer posición (es decir, comuna_1[1]) no puede ser contagiado porque sus vecinos no están infectados. En cambio, el vecino en la ultima posición (es decir, comuna_1[-1]) si puede ser contagiado porque su vecino está infectado)).

El largo de la lista será igual al total de ciudadanos en dicha comuna. Y la cantidad de 'S' en la lista será el numero de personas susceptibles en esa comuna, y la cantidad de 'I0' en la lista será el numero de personas infectadas esta semana, y así sucesivamente con las 4 letras ('S', I0', 'I1', 'R') que catalogan el estado de las personas de la comuna.



In [ ]:
comunas = {}

for sir_com in sir_por_comuna.keys():
    comuna = []
    for status, n in sir_por_comuna[sir_com].items(): 
        if status == 'N':
            continue
        else:
             #TU CODIGO ACA
            
    random.shuffle(comuna)
    comunas[sir_com] = comuna


A continuación impriman una comuna para ver como les quedó la representación.

In [ ]:
print(comunas['2'])

#### Ejercicio 6:

Construyan una función que cuente la cantidad personas de cierto tipo en una comuna. Esta es una función auxiliar que ahora puede ayudarlos a verificar si el ejercicio 5 construyó bien las comunas. Pero además es una función que cumplirá un rol importante en la simulación final.

In [ ]:
def contar_tipo(comuna, tipo='I0'):
    '''
    Esta funcion cuenta la cantidad de personas del tipo especificado en una 
    comuna
    
    Inputs:
        comuna (lista): el estado de todas las personas en la comuna
        tipo (str): indica si se quiere contar los S (suceptibles), I0 
                    (infectados esta semana), I1 (infectados hace una semana),
                    o R (recuperados).
    Output 
        personas_del_tipo (int): personas del tipo seleccionado
    '''
    personas_del_tipo = 0
    
    #TU CODIGO ACA
    
    return personas_del_tipo

Prueben si su función realiza la tarea que ustedes querían de la siguiente forma (pueden probar con otros tipos y otras comunas también):

In [ ]:
contar_tipo(comunas['1'], tipo='I0') == sir_por_comuna['1']['I0']

#### Ejercicio 7:
Creen una funcion que se llame 'es_infectado'. Esta función determinará si una persona que tiene un vecino en estado 'I0' o 'I1' es infectado o no. Para ello, la función tendrá como argumentos la comuna analizada, la posición del inidividuo analizado y la tasa de contagio. 

Además, a cada individuo le asignaremos un valor random de inmunidad al momento de analizarlo (usen random.random() para crear un valor que representará que tan fuerte esta su sistema inmunológico). Este valor de inmunidad lo compararemos con la tasa de contagio. Si es menor que la tasa_de_contagio, resultará infectado, sino no.

Notas: 
- Recuerden que la primer posición de una lista es la 0. 
- En esta simulacion asumiremos que la persona en la primer posición solo se podra contagiar de la persona en la posición 1 (porque no tiene vecinos a su izquierda) y que la persona en la ultima posición solo se podra contagiar de la persona en la anteúlitma posición (porque no tiene vecinos a su izquierda) 

In [ ]:
def es_infectado(comuna, posicion, tasa_de_contagio):
    '''
    Determinar si una persona es infectada o no.
    
    Inputs:
        comuna (list): el estado de todas las personas de la comuna al comienzo 
                       de la semana. 
        posicion (int): la posicion del inidividuo analizado 
        tasa_de_contagio (float): la probabilidad de ser infectado dado que tu 
                                  vecino está infectado.
        
    Output:
         True, si la persona se infectó, False caso contrario.
    '''
    
    #TU CODIGO ACA


#### Ejercicio 8:
A continuación, simulen el paso de una semana. Al cabo de una semana, las personas que tenían estado I0 pasaran a I1, los de estado I1 pasaran a estar recuperados y los de estado R continuaran como recuperados. Para quienes estaban en estado S, debemos aplicarles la función es_infectado para saber si se contagiará o no al comienzo de esta nueva semana.

Esta función se debe llamar simular_una_semana() y sus atributos serán: la comuna a simular (es decir la lista que contiene el estado de todos los ciudadanos de la comuna) y la tasa de contagio. La función debe devolver una nueva lista con el estado de todas las personas de la comuna al comienzo de la semana t+1.


In [ ]:
def simular_una_semana(comuna_s1, tasa_de_contagio):
    '''
    Mover la simulacion un dia. 

    Inputs:
        comuna_s1 (list): el estado de todas las personas de la comuna al comienzo 
            de la semana. 
        tasa_de_contagio (float): la probabilidad de ser infectado dado que tu 
            vecino está infectado.
        
    Output:
        comuna_s2 (list): el estado de todas las personas de la comuna al 
            comienzo de la semana t+1. 
    '''
    comuna_s2 = []

    #TU CODIGO ACA
    
    return comuna_s2



#### Ejercicio 9:
Para finalizar creen una función para correr la simulación durante varias semanas. Su función se debe llamar correr_simulacion y debe tomar como argumentos el estado inicial de la comuna, la semilla aleatoria, el número máximo de semanas para simular (max_num_semanas) y la tasa de contagio. Además, debe devolver una tupla con el estado final de la comuna y el número de semanas simuladas (s).

##### Notas:

- max_num_semanas siempre debe ser mayor a 0. Es decir que su función debe ejecutar como mínimo la simulación de una semana completa. 

- Antes de la simulación de la 1er semana se debe fijar la semilla para la función random.

- Su simulación debe comenzar la semana 0 y contar el número de semanas simuladas. Por ejemplo, si su simulación comienza la semana 0 y alcanza las condiciones de finalización después de simular una semana, debería devolver 1 como el número de semanas. Por otro lado, si su simulación comienza la semana 0 y se ejecuta para la semana 0 y la semana 1, debería devolver 2 como el número de semanas simuladas.

- Recuerden que hay dos condiciones para dejar de correr la simulación: 

    a) que hayan pasado la cantidad de semanas fijadas en max_num_semanas o 

    b) que nadie en la comuna está infectado después de simular una semana determinada. Deben usar la función contar_tipo para verificar esta condición y debe verificar esta condición después de simular una semana (¡no antes!).


In [ ]:
TASA_CONTAGIO = 0.5
SEMILLA = 5000

def correr_simulacion(comuna, max_num_semanas, semilla=SEMILLA, 
                      tasa_de_contagio=TASA_CONTAGIO):
    '''
    Correr la simulacion para todas las semanas (con el limite de tiempo 
    definido en max_num_semanas)

    Inputs:
        comuna (list): el estado de todas las personas de la comuna al 
            comienzo de la semana. 
        semilla (int): el numero inicial para la funcion random que se usará 
            para la simulación
        max_num_semanas (int): la maxima cantidad de semanas a simular
        tasa_de_contagio (float): la probabilidad de ser infectado dado que tu 
            vecino está infectado.

    Output:
        tuple (s, comuna_s2) of
            s (int): semanas de simulación
            comuna_s2 (list): el estado de todas las personas de la comuna al 
                comienzo de la semana s.    
    '''
    #Nos aseguramos de que el numero de semanas a simular sea positivo:
    assert max_num_semanas > 0
    
    #Fijamos la semilla:
    random.seed(semilla)
     
    #TU CODIGO ACA
            
    return (s, comuna_s)


Para probar si la simulacion les salió bien corranla para una comuna variando la tasa de contagio. Cuanto más baja la tasa menos personas llegan a ser contagiadas y en menos semanas tenemos a toda la población en estado 'S' o 'R'